In [34]:
# import argparse
import os
from epitome.models import *
from epitome.functions import *
from epitome.viz  import *

from epitome.constants import *
# import yaml
# import subprocess
from timeit import default_timer as timer
import numpy
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pybedtools

In [2]:
# Data directories
motif_dir = "data/motif_data"
epitome_dir = "data/epitome_data/"
feature_path = os.path.join(epitome_dir, "feature_name")

In [3]:
# Load in data for epitome
train_data = scipy.sparse.load_npz(os.path.join(epitome_dir, 'train.npz')).toarray()
valid_data = scipy.sparse.load_npz(os.path.join(epitome_dir, 'valid.npz')).toarray()
test_data = scipy.sparse.load_npz(os.path.join(epitome_dir, 'test.npz')).toarray()
data = {Dataset.TRAIN: train_data, Dataset.VALID: valid_data, Dataset.TEST: test_data}

regions_file = os.path.join(epitome_dir, POSITIONS_FILE)
chip_data = concatenate_all_data(data, regions_file)

In [4]:
# Original matrix and data
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = None,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
# matrix[[cellmap], [assaymap]] --> better to get feature names from here

In [5]:
assaymap.keys(), len(set(assaymap.keys()))

(dict_keys(['DNase', 'ZZZ3', 'ZSCAN29', 'ZNF592', 'ZNF384', 'ZNF274', 'ZNF24', 'ZNF217', 'ZNF207', 'ZNF143', 'ZKSCAN1', 'ZHX1', 'ZFX', 'ZFP36', 'ZBTB7A', 'ZBTB40', 'ZBTB33', 'YY1', 'YBX1', 'USF2', 'USF1', 'UBTF', 'TRIM22', 'TOE1', 'TEAD4', 'TCF7L2', 'TCF7', 'TCF12', 'TBP', 'TBL1XR1', 'TARDBP', 'TAF7', 'TAF1', 'SUZ12', 'STAT5A', 'STAT3', 'STAT1', 'SRF', 'SREBF2', 'SREBF1', 'SPI1', 'SP2', 'SP1', 'SMC3', 'SMARCE1', 'SMARCC2', 'SMARCB1', 'SMARCA5', 'SMARCA4', 'SMAD5', 'SIX5', 'SIN3B', 'SIN3A', 'SAP30', 'RXRA', 'RNF2', 'RFX5', 'RFX1', 'REST', 'RCOR1', 'RBBP5', 'RAD51', 'RAD21', 'POU2F2', 'POLR2AphosphoS5', 'POLR2AphosphoS2', 'POLR2A', 'PML', 'PKNOX1', 'PHF8', 'PAX5', 'NRF1', 'NR3C1', 'NR2C2', 'NONO', 'NFYB', 'NFYA', 'NFXL1', 'NFRKB', 'NFIC', 'NFE2L2', 'NFATC3', 'NCOR1', 'NBN', 'MYC', 'MXI1', 'MTA3', 'MTA2', 'MTA1', 'MNT', 'MLLT1', 'MEF2A', 'MAZ', 'MAX', 'MAFK', 'MAFF', 'LEF1', 'LARP7', 'KDM5A', 'KDM1A', 'JUND', 'JUN', 'IRF3', 'IKZF1', 'HSF1', 'HDGF', 'HDAC6', 'HDAC2', 'HDAC1', 'HCFC1', 'GTF

In [6]:
motif_data_dir = "/data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut"
f = []
for (dirpath, dirnames, filenames) in os.walk(motif_data_dir):
    tfs = [file.split("_")[0] for file in filenames]
    f.extend(tfs)
    break

In [11]:
# Epitome TF's that we have data for in HOCOMOCCO
tfs_hoco = set(assaymap.keys()).intersection(set(f))
print(len(tfs_hoco))
files_tfs_hoco = []
for (dirpath, dirnames, filenames) in os.walk(motif_data_dir):
    for filename in filenames:
        tf = filename.split("_")[0]
        if tf in tfs_hoco and "sorted" in filename:
            files_tfs_hoco.append(filename)
print(len(files_tfs_hoco))

66
83


In [47]:
#pd.Series(list(tfs_hoco)).rename("TF").to_csv("data/motif_data/Epitome_HOCOMOCCO_TF.csv")
queried_tfs = pd.read_csv("data/motif_data" + "/Epitome_HOCOMOCCO_TF.csv")['TF'] #parser.parse_args().tf
# queried_tfs.to_list()

In [37]:
a = pybedtools.BedTool('data/epitome_data/all.pos.bed')
a.count()

3268840

In [46]:
! python generate_motif_matrix.py --tf garb

/home/eecs/jahnavis/epitome_new/epitome-1/epitome
['NFYA', 'NFYB', 'ATF3', 'CEBPZ', 'MAZ', 'HSF1', 'ZFX', 'STAT3', 'NRF1', 'GATA3', 'E2F1', 'MXI1', 'USF2', 'EGR1', 'TCF7', 'ESR1', 'SRF', 'TBP', 'JUN', 'BACH1', 'PAX5', 'FOXA1', 'MYC', 'NFIC', 'E2F8', 'BRCA1', 'ETV6', 'NR2C2', 'TAF1', 'IKZF1', 'MAFF', 'E4F1', 'CUX1', 'RXRA', 'SP2', 'ETS1', 'LEF1', 'GABPA', 'E2F6', 'ATF2', 'ARNT', 'RFX5', 'MAX', 'SPI1', 'SP1', 'IRF3', 'ELK1', 'USF1', 'ATF7', 'STAT1', 'FOSL2', 'FOXM1', 'JUND', 'ELF1', 'CEBPB', 'TEAD4', 'MAFK', 'CREM', 'RFX1', 'CREB1', 'FOS', 'FOSL1', 'REST', 'E2F4', 'MEF2A', 'CTCF']
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/USF1_HUMAN.H11MO.0.A_co1331_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/FOSL2_HUMAN.H11MO.0.A_co1484_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOu

Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/E4F1_HUMAN.H11MO.0.D_co1354_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/GABPA_HUMAN.H11MO.0.A_co1313_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/RXRA_HUMAN.H11MO.0.A_co1259_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/HSF1_HUMAN.H11MO.0.A_co1404_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/FOSL1_HUMAN.H11MO.0.A_co1462_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/geneInductionPipeline/TF_motifs/HOCOMOCO/pwmscanOut/ETV6_HUMAN.H11MO.0.D_co1273_matrix_scan.bed.sorted.bed ...
Looking at /data/yosef2/DC/footprintPrediction/gen

In [87]:
# TODO: Add to original matrix generator script
motif_map = pd.read_csv(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_motifmap.csv"), header=None).rename(
    columns={0:"Index", 1:"TF"})
unique_tf = motif_map["TF"].unique()

motif_mat_sum = np.empty((0, 3268840))
motif_map_sum = []

for tf in unique_tf:
    tf_index = motif_map[motif_map["TF"]==tf]["Index"]
    tf_motif = motif_mat[tf_index,:]
    tf_motif_sum = np.sum(tf_motif, axis=0)
    tf_motif_sum[tf_motif_sum > 0] = 1
    motif_mat_sum = np.append(motif_mat_sum, [tf_motif_sum], axis=0)
    motif_map_sum = np.append(motif_map_sum, tf)

motif_mat_sum.shape

(66, 3268840)

In [97]:
motifs_0 = motif_mat_sum[:,1]
print(np.unique(motifs_0))
motifs_0[motifs_0 > 0] = 1
print(np.unique(motifs_0))

[0. 1.]
[0. 1.]


In [56]:
motif_mat = np.load(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_motifmat.npz"))["tf"]
motif_mat.shape

(83, 3268840)

In [86]:
motif_mat[[1,3,5],:].shape

(3, 3268840)

In [6]:
# Load motif data
motif_mat = np.load(os.path.join(motif_dir, "CTCF_HOCOMOCO_motifmat.npz"))['tf'][0]
motif_map = pd.read_csv(os.path.join(motif_dir, "CTCF_HOCOMOCO_motifmap.csv"), header=None)

In [11]:
chip_k562_ctcf = chip_data[matrix[cellmap['K562'], assaymap['CTCF']],:]
chip_k562_ctcf.shape

(3268840,)

In [13]:
chip_k562_dnase = chip_data[matrix[cellmap['K562'], assaymap['DNase']],:]
chip_k562_dnase.shape

(3268840,)

In [14]:
jaccard_score(chip_k562_ctcf, motif_mat)

0.18570827671002393

In [23]:
jaccard_ctcf_motif = {}
for cell_type in cellmap.keys():
    score = jaccard_score(chip_data[matrix[cellmap[cell_type], assaymap['CTCF']],:], motif_mat)
    jaccard_ctcf_motif[cell_type] = score
    print("{}: {}".format(cell_type, score))

T47D: 0.16931983890007637
SK-N-SH: 0.19625284755691239
MCF-7: 0.17406631815237206
K562: 0.18570827671002393
Ishikawa: 0.1503725877332654
IMR-90: 0.18723004998624973
HepG2: 0.19004735964073088
HeLa-S3: 0.18861978591051184
HEK293T: 0.0375234806944962
HCT116: 0.19425196792412538
H1: 0.12943820990072033
GM12892: 0.15670631349370434
GM12891: 0.1395153571008749
GM12878: 0.18085123697916666
A549: 0.16926544240400668


In [20]:
jaccard_dnase = {}
for cell_type in cellmap.keys():
    score = jaccard_score(chip_data[matrix[cellmap[cell_type], assaymap['CTCF']],:], 
                          chip_data[matrix[cellmap[cell_type], assaymap['DNase']],:])
    jaccard_dnase[cell_type] = score
    print("{}: {}".format(cell_type, score))

T47D: 0.06051556175314237
SK-N-SH: 0.10359291547806304
MCF-7: 0.10466254516362974
K562: 0.09362020812484875
Ishikawa: 0.1162066628816322
IMR-90: 0.09688068091061908
HepG2: 0.0934063954577551
HeLa-S3: 0.12387221016627624
HEK293T: 0.08565332894928579
HCT116: 0.20921013484718123
H1: 0.048866313548055956
GM12892: 0.10636024698271415
GM12891: 0.11102015842304576
GM12878: 0.08846546554309052
A549: 0.09218944857127366


In [21]:
chip_all_ctcf = chip_data[matrix[:, assaymap['CTCF']],:]
chip_all_ctcf_sum = np.sum(chip_all_ctcf, axis=0)
chip_all_ctcf_sum[chip_all_ctcf_sum > 0] = 1

In [22]:
jaccard_score(chip_all_ctcf_sum, motif_mat)

0.1985260701344961

In [32]:
np.mean(list(jaccard_ctcf_motif.values()))

0.16327793820581576